<a href="https://colab.research.google.com/github/MarceloCCortez/Proyecto/blob/master/Clasificador%20de%20im%C3%A1genes%20con%20Python%20y%20Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
datos, metadatos = tfds.load('fashion_mnist',as_supervised=True,with_info=True)

In [ ]:
metadatos

In [ ]:
datos_entrenamiento, datos_prueba = datos['train'], datos['test']

In [ ]:
nombres_clases =  metadatos.features['label'].names


In [ ]:
nombres_clases

In [ ]:
#normalizar los datos (pasar de 0-255 a 0-1)
def normalizar(imagenes,etiquetas):
  imagenes = tf.cast(imagenes,tf.float32)
  imagenes /= 255
  return imagenes, etiquetas

#Normalizar los datos de entrenamiento y pruebas con la funcion que hicimos 
datos_entrenamiento = datos_entrenamiento.map(normalizar)
datos_prueba = datos_prueba.map(normalizar)
#Agregar cache
datos_entrenamiento = datos_entrenamiento.cache()
datos_prueba = datos_prueba.cache()


In [ ]:
#Mostrar imagenes de los datos de prueba
for imagen, etiqueta in datos_entrenamiento.take(1):
  break
imagen = imagen.numpy().reshape((28,28)) #Redimensionar
import matplotlib.pyplot as plt
#Dibujar 
plt.figure()
plt.imshow(imagen,cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i,(imagen,etiqueta) in enumerate(datos_entrenamiento.take(25)):
  imagen = imagen.numpy().reshape((28,28))
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(imagen,cmap=plt.cm.binary)
  plt.xlabel(nombres_clases[etiqueta])
plt.show()

In [ ]:
#Crear modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dense(50,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax) #Para darle la clasificacion 
])

In [ ]:
#compilar el modelo
modelo.compile(
    optimizer= 'adam',
    loss= tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
    )

In [ ]:
num_ej_entrenamiento= metadatos.splits['train'].num_examples
num_ej_pruebas = metadatos.splits['test'].num_examples

In [ ]:
print(num_ej_entrenamiento)
print(num_ej_pruebas)

In [ ]:
TAMANO_LOTE = 32
datos_entrenamiento = datos_entrenamiento.repeat().shuffle(num_ej_entrenamiento).batch(TAMANO_LOTE)
datos_prueba = datos_prueba.batch(TAMANO_LOTE)

In [ ]:
import math 
#entrenar
historial = modelo.fit(datos_entrenamiento,epochs=5,steps_per_epoch=math.ceil(num_ej_entrenamiento/TAMANO_LOTE))

In [ ]:
plt.xlabel('# Epoca')
plt.ylabel('Magnitud de perdida')
plt.plot(historial.history['loss'])

In [ ]:
import numpy as np

for imagenes_prueba, etiquetas_prueba in datos_prueba.take(1):
  imagenes_pruebas = imagenes_prueba.numpy()
  etiquetas_prueba = etiquetas_prueba.numpy()
  predicciones = modelo.predict(imagenes_prueba)
def graficar_imagen(i,arr_predicciones,etiquetas_reales,imagenes):
  arr_predicciones, etiqueta_real, img = arr_predicciones[i], etiquetas_reales[i], imagenes[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img[...,0],cmap=plt.cm.binary)
  etiqueta_prediccion = np.argmax(arr_predicciones)
  if etiqueta_prediccion == etiqueta_real:
    color = 'blue' # Si le atino
  else:
    color = 'red' #OOps no le atino
  plt.xlabel("{}{:2.0f}% ({})".format(
      nombres_clases[etiqueta_prediccion],
      100*np.max(arr_predicciones),
      nombres_clases[etiqueta_real],
      color=color
  ))
def graficar_valor_arreglo(i,arr_predicciones,etiqueta_real):
  arr_predicciones,etiqueta_real = arr_predicciones[i],etiqueta_real[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  grafica = plt.bar(range(10),arr_predicciones, color= "#777777")
  plt.ylim([0,1])
  etiqueta_prediccion=np.argmax(arr_predicciones)
  grafica[etiqueta_prediccion].set_color('red')
  grafica[etiqueta_real].set_color('blue')
filas = 5
columnas = 5
num_imagenes = filas*columnas
plt.figure(figsize=(2*2*columnas,2*filas))
for i in range(num_imagenes):
  plt.subplot(filas,2*columnas,2*i+1)
  graficar_imagen(i,predicciones,etiquetas_prueba,imagenes_prueba)
  plt.subplot(filas,2*columnas,2*i+2)
  graficar_valor_arreglo(i,predicciones,etiquetas_prueba)

In [ ]:
#Probar una imagen suelta
imagen = imagenes_prueba[4] #AL ser la variable imagenes_prueba solo tiene lo que se le puso en el bloque anterior heheh
imagen = np.array([imagen])
prediccion = modelo.predict(imagen)

print("Prediccion: " + nombres_clases[np.argmax(prediccion[0])])